# Import libraries

In [1]:
!pip install sklearn
!pip install pandas
!pip install matplotlib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []

# parameters
steps = 96
n_hidden = 1
units = 150
batch_size = 96
epochs = 100

# months to evaluate model on
date = 2018070000

# for later use
features_num = 14

In [2]:
data = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values in the whole data set
data.fillna(data.mean(), inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Required functions to put data into required shape for LSTM:

In [3]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# Set up X and y for train, test and val into correct shapes

In [4]:
# divide features and labels
X_train = data_train[:, 0:14] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:14]
y_test = data_test[:, -1] 

# divide data into validation and normal test 
X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

# cut data
X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# Import libraries:

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

# Design LSTM:

In [6]:
def regressor_tunning(kernel_initializer = 'he_uniform',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 0:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop()
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model

model = regressor_tunning()

# Train the model:

In [7]:
history = model.fit(X_train,
                    y_train, 
                    batch_size = batch_size, 
                    epochs = epochs,
                    shuffle = False, 
                    validation_data = (X_val, y_val))
    
# required before predicitons
model.reset_states()
    
y_pred = model.predict(X_test, batch_size = batch_size)

Epoch 1/100
65/65 [==============================] - 50s 774ms/step - loss: 0.3067 - mse: 0.3067 - mae: 0.3756 - val_loss: 0.0058 - val_mse: 0.0058 - val_mae: 0.0556
Epoch 2/100
65/65 [==============================] - 39s 606ms/step - loss: 0.0811 - mse: 0.0811 - mae: 0.2261 - val_loss: 0.0293 - val_mse: 0.0293 - val_mae: 0.1689
Epoch 3/100
65/65 [==============================] - 40s 615ms/step - loss: 0.0289 - mse: 0.0289 - mae: 0.1339 - val_loss: 0.0023 - val_mse: 0.0023 - val_mae: 0.0423
Epoch 4/100
65/65 [==============================] - 41s 631ms/step - loss: 0.0106 - mse: 0.0106 - mae: 0.0806 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0274
Epoch 5/100
65/65 [==============================] - 45s 699ms/step - loss: 0.0049 - mse: 0.0049 - mae: 0.0543 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0195
Epoch 6/100
65/65 [==============================] - 51s 792ms/step - loss: 0.0026 - mse: 0.0026 - mae: 0.0382 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0176
Epoc

Epoch 48/100
65/65 [==============================] - 59s 903ms/step - loss: 7.4017e-04 - mse: 7.4017e-04 - mae: 0.0143 - val_loss: 9.7035e-04 - val_mse: 9.7035e-04 - val_mae: 0.0145
Epoch 49/100
65/65 [==============================] - 57s 870ms/step - loss: 7.3133e-04 - mse: 7.3133e-04 - mae: 0.0141 - val_loss: 9.4491e-04 - val_mse: 9.4491e-04 - val_mae: 0.0145
Epoch 50/100
65/65 [==============================] - 50s 767ms/step - loss: 7.3105e-04 - mse: 7.3105e-04 - mae: 0.0141 - val_loss: 9.4495e-04 - val_mse: 9.4495e-04 - val_mae: 0.0146
Epoch 51/100
65/65 [==============================] - 50s 766ms/step - loss: 7.2916e-04 - mse: 7.2916e-04 - mae: 0.0140 - val_loss: 9.4975e-04 - val_mse: 9.4975e-04 - val_mae: 0.0145
Epoch 52/100
65/65 [==============================] - 49s 759ms/step - loss: 7.2943e-04 - mse: 7.2943e-04 - mae: 0.0140 - val_loss: 9.1885e-04 - val_mse: 9.1885e-04 - val_mae: 0.0147
Epoch 53/100
65/65 [==============================] - 53s 814ms/step - loss: 7.2923e-

Epoch 93/100
65/65 [==============================] - 45s 692ms/step - loss: 6.5494e-04 - mse: 6.5494e-04 - mae: 0.0130 - val_loss: 9.8416e-04 - val_mse: 9.8416e-04 - val_mae: 0.0143
Epoch 94/100
65/65 [==============================] - 43s 664ms/step - loss: 6.5852e-04 - mse: 6.5852e-04 - mae: 0.0131 - val_loss: 9.1539e-04 - val_mse: 9.1539e-04 - val_mae: 0.0140
Epoch 95/100
65/65 [==============================] - 44s 675ms/step - loss: 6.5859e-04 - mse: 6.5859e-04 - mae: 0.0130 - val_loss: 8.8259e-04 - val_mse: 8.8259e-04 - val_mae: 0.0143
Epoch 96/100
65/65 [==============================] - 47s 722ms/step - loss: 6.5728e-04 - mse: 6.5728e-04 - mae: 0.0130 - val_loss: 9.0433e-04 - val_mse: 9.0433e-04 - val_mae: 0.0144
Epoch 97/100
65/65 [==============================] - 42s 651ms/step - loss: 6.5721e-04 - mse: 6.5721e-04 - mae: 0.0130 - val_loss: 9.5625e-04 - val_mse: 9.5625e-04 - val_mae: 0.0147
Epoch 98/100
65/65 [==============================] - 35s 536ms/step - loss: 6.4362e-

# Evaluate predictions:

In [8]:
# cannot use inverse function; prices col = 14
y_pred = (y_pred * sc_X.data_range_[14]) + (sc_X.data_min_[14])
y_test = (y_test * sc_X.data_range_[14]) + (sc_X.data_min_[14])

# Reshaping
y_pred = np.reshape(y_pred, (y_pred.shape[0]))

# =============================================================================
# METRICS EVALUATION (1) for the whole test set
# =============================================================================
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

# calculate metrics
rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)

# append to list
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)

# Need to process data with spike occurences in the same way as features and offers:
(Required to evaluate predictions in both normal regions and spike regions)

In [9]:
# download data for shaded area
data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# make sure shaded area will correspond to values outputed by LSTM
data.reset_index(drop = True, inplace = True)

# fill_nan is already made - so lets split data into test and train
from sklearn.model_selection import train_test_split

# divide data into train and test 
shade_train, shade_test = train_test_split(
         data, test_size = 0.15, shuffle = False)

# reset index of testing data
shade_test.reset_index(drop = True, inplace = True)

# function to split data into correct shape for RNN
def split_data(shade_test, steps):
    y_spike_occ = list()
    upper_lim = list()
    lower_lim = list()
    for i in range(steps, len(shade_test.index)):
        y_spike_occ.append(shade_test['spike_occurance'][i])
        upper_lim.append(shade_test['spike_upperlim'][i])
        lower_lim.append(shade_test['spike_lowerlim'][i])
    return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

y_spike_occ, spike_upperlim, spike_lowerlim = split_data(shade_test, steps)
y_spike_occ = cut_data(y_spike_occ, batch_size)
spike_upperlim = cut_data(spike_upperlim, batch_size)
spike_lowerlim = cut_data(spike_lowerlim, batch_size)

# Evaluate on spike and normal regions

In [10]:
# =============================================================================
# METRICS EVALUATION (2) on spike regions
# =============================================================================

# smal adjustment
#y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)

# append ot lists
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# METRIC EVALUATION (3) on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)

# append to list
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)

# Now evaluate predictions on Spike & Normal regions:

In [11]:
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

# Save y_pred & spike limits to plot with Stateless LSTM
datasave = pd.DataFrame({'y_pred': y_pred,
                         'y_test':y_test,
                         'spike_upperlim':spike_upperlim,
                         'spike_lowerlim':spike_lowerlim})

datasave.to_csv('LSTM_plot_Stateful_6m.csv')

In [12]:
results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,29.129889,17.739845,58.859746,44.155569,21.092452,13.663671
